In [1]:
import os

In [2]:
%pwd

'c:\\Users\\NARINDER\\Desktop\\Youtube Project\\research'

In [3]:
os.chdir('..')

In [4]:
%pwd

'c:\\Users\\NARINDER\\Desktop\\Youtube Project'

In [5]:
import mlflow

In [6]:
import tensorflow as tf

model = tf.keras.models.load_model('artifacts/training/model.h5')

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [8]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifact_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config


In [20]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from mlflow.models.signature import infer_signature


class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            class_mode="sparse",
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path_to_json=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_tracking_uri("databricks://DEFAULT")
        mlflow.set_experiment("/Users/tanwaraditya2004@gmail.com/TumorDetection")

        with mlflow.start_run():
            # Prepare a batch for signature inference
            X_batch, _ = next(self.valid_generator)
            y_pred = self.model.predict(X_batch)
            signature = infer_signature(X_batch, y_pred)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({
                "loss": self.score[0],
                "accuracy": self.score[1]
            })
            mlflow.log_artifact("scores.json")

            # Log and register model in one step
            mlflow.set_registry_uri("databricks-uc")

            mlflow.keras.log_model(
                self.model,
                name="model",
                registered_model_name="prod.ml_models.MyModel",
                signature=signature
            )

In [21]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

Found 139 images belonging to 2 classes.


c:\Users\NARINDER\Desktop\Youtube Project\project\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 ━━━━━━━━━━━━━━━━━━━━ 24s 3s/step - accuracy: 0.4820 - loss: 25.7008
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


Registered model 'prod.ml_models.MyModel' already exists. Creating a new version of this model...
c:\Users\NARINDER\Desktop\Youtube Project\project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Uploading artifacts: 100%|██████████| 6/6 [02:55<00:00, 29.26s/it]
Created version '1' of model 'prod.ml_models.mymodel'.


🏃 View run puzzled-dog-544 at: https://dbc-b60deabe-a70d.cloud.databricks.com/ml/experiments/2526556676783277/runs/0c0871120d844ba3ab3699b357fb5bb5
🧪 View experiment at: https://dbc-b60deabe-a70d.cloud.databricks.com/ml/experiments/2526556676783277
